convert qupath annotation to imagescope xml in order to train deeplab model in MATLAB
210421 last edit

In [1]:
from paquo.projects import QuPathProject
import numpy as np
import os
import xml.etree.ElementTree as ET
from xml.dom import minidom
import cv2
from matplotlib import pyplot as plt

In [2]:
qppath = r"\\motherserverdw\Kyu_Sync\Research\Active\Aging_organized\svs\svs_back\qupath_v2\project.qpproj"
with QuPathProject(qppath, mode='r+') as qp:
    if not os.path.exists(qp.images[0].uri[7:]):
        if os.path.exists(r"\\motherserverdw\Kyu_Sync"):
            for image in qp.images:
                qp.update_image_paths(uri2uri={image.uri: image.uri.replace('kukissd','10.162.80.6')})
        elif os.path.exists(r"\\kukissd\Kyu_Sync"):
            for image in qp.images:
                qp.update_image_paths(uri2uri={image.uri: image.uri.replace('10.162.80.6','kukissd')})
        else:
            raise NameError('server is not accessible')

KeyboardInterrupt: 

In [3]:
for image in qp.images:
    print(image.image_name)
    coords = []
    #read annotation
    annotations = image.hierarchy.annotations
    for annotation in annotations:
        if not annotation.path_class is None:
            poly = annotation.roi
            if poly.type=='MultiPolygon':
                # coords.append(np.concatenate(tuple([np.array(x.exterior.coords) for x in poly.geoms])).astype(np.int32))
                while poly.type=='MultiPolygon':
                    poly = poly.buffer(0.0001)
                coords.append(np.array(poly.exterior.coords).astype(np.int32))
            if poly.type=='Polygon':
                coords.append(np.array(poly.exterior.coords).astype(np.int32))
            if poly.type=='LineString':
                coords.append(np.array(poly.coords).astype(np.int32))
    coords=np.array(coords)
    names =np.array(names)
    root = ET.Element("Annotations", MicronsPerPixel="0.230500")
    classes = [_.name for _ in qp.path_classes][1:]
    for idx,elem in enumerate(classes):
        #basic setup
        m1 = ET.Element("Annotation", Id=str(idx+1), Name="", ReadOnly="0",
                        NameReadOnly="0", LineColorReadOnly="0",
                        Incremental="0", Type="4", LineColor="16711935",
                        Visible="1", Selected="0", MarkupImagePath="", MacroName="")
        root.append(m1)
        b1 = ET.SubElement(m1, "Attributes")
        c1 = ET.SubElement(b1, "Attribute", Name=elem, Id="0", Value="")
        b2 = ET.SubElement(m1, "Regions")
        c2 = ET.SubElement(b2, "RegionAttributeHeaders")
        ET.SubElement(c2, "AttributeHeader", Id="9999", Name="Region", ColumnWidth="-1")
        ET.SubElement(c2, "AttributeHeader", Id="9997", Name="Length", ColumnWidth="-1")
        ET.SubElement(c2, "AttributeHeader", Id="9996", Name="Area", ColumnWidth="-1")
        ET.SubElement(c2, "AttributeHeader", Id="9998", Name="Text", ColumnWidth="-1")
        ET.SubElement(c2, "AttributeHeader", Id="1", Name="Description", ColumnWidth="-1")
        # individual annotation circle
        for annidx,annelem in enumerate(coords[names==elem]):
            d2 = ET.SubElement(b2, "Region", Id=str(annidx+1), Type="0", Zoom="1", Selected="0", ImageLocation="", ImageFocus="-1",
            Text="", NegativeROA="0", InputRegionId="0", Analyze="1",DisplayId=str(annidx+1))
            ET.SubElement(d2,"Attributes")
            e2 = ET.SubElement(d2,"Vertices")
            for vertidx,vertelem in enumerate(annelem):
                ET.SubElement(e2,"Vertex", X=str(vertelem[0]), Y=str(vertelem[1]), Z="0")
        b3 = ET.SubElement(m1, "Plots")
        xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent="   ")
    with open(image.image_name.replace('svs','xml'),'wb') as ff:
        ff.write(xmlstr.encode('utf-8'))

wf16_b2_01.svs


<ipython-input-3-b309e324b184>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  coords=np.array(coords)


In [33]:
# xmin,ymin = np.min(coords[0],axis=0)
# coords[0][:,0]=coords[0][:,0]-xmin
# coords[0][:,1]=coords[0][:,1]-ymin
# im = np.zeros(np.max(coords[0],axis=0),dtype=np.int8)
# cv2.fillPoly(im, [coords[0]], 1)
# im.astype(bool)
# plt.imshow(im)
